Created by: [SmirkyGraphs](https://smirkygraphs.github.io/). Code: [Github](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [HHS](https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/anag-cw7u).
<hr>

# Rhode Island Hospital Staffing Shortage

In December 2021, hospital leaders have warned of [a crisis](https://www.wpri.com/health/coronavirus/an-unprecedented-crisis-at-hospitals-in-ri-amid-covid-surge-staff-shortage/) and overcrowding. While neither RIDOH, or the hospitals themselves release icu/total beds available/used, HHS tracks many metrics that can be used to gage staffing in individual hospitals. This notebook will focus specifically on the largest, being Rhode Island Hospital.

Starting at the end of July 2020 HHS started collecting weekly numbers on hospitals regarding beds/staffing and covid numbers weekly. This notebook will collect data from HHS's "COVID-19 Reported Patient Impact and Hospital Capacity by Facility" dataset to pull weekly average number of ICU and inpatent beds.

<hr>

In [1]:
import pandas as pd

def scrape_hhs_data(state):
    # https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/anag-cw7u # prepped
    # https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/uqq2-txqb # raw
    url = f'https://healthdata.gov/resource/anag-cw7u.csv?state={state}' # hospital-level data
    df = pd.read_csv(url, parse_dates=['collection_week'])
    df = df.replace(-999999, None) # -999999 = "same as ri's small number policy"
    df.to_csv(f'./data/raw/state/{state}-hhs-hospital-level.csv', index=False)

    # https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/g62h-syeh 
    url = f'https://healthdata.gov/resource/g62h-syeh.csv?state={state}' # state-level data
    df = pd.read_csv(url, parse_dates=['date']).sort_values(by='date')

    print('[status] hhs data collected')
    df.to_csv(f'./data/raw/state/{state}-hhs-state-level.csv', index=False)

In [2]:
scrape_hhs_data('RI')

[status] hhs data collected


In [3]:
df = pd.read_csv('./data/raw/state/RI-hhs-hospital-level.csv', parse_dates=['collection_week'])

cols = [
    'total_icu_beds_7_day_avg',
    'inpatient_beds_7_day_avg'
]

df = df.groupby(['hospital_name', 'collection_week'])[cols].sum().reset_index()
df = df[df['hospital_name'].str.contains('RHODE ISLAND HOSPITAL')]

In [4]:
df.to_csv('./data/clean/ri-hospital-beds.csv', index=False)